In [1]:
import pennylane as qml
import numpy as np

In [ ]:
H = 1.0 / np.sqrt(2) * np.array([
    [1, 1],
    [1, -1]
], dtype=complex)

def get_sqp(probabilities):
    qubit_count = int(np.log2(len(probabilities)))
    sqp = []
    for i in range(qubit_count):
        sum = 0.0
        for j in range(len(probabilities)):
            if f"{j:08b}"[::-1][i] == '1':
                sum += probabilities[j]
        sqp += [sum]
    sqp.reverse()
    return np.array(sqp)

In [ ]:
shots_list = None

qubit_count = 5
solution = "10110"

dev = qml.device("default.qubit", wires=qubit_count+1, shots=shots_list)

@qml.qnode(dev)
def grovers(qubit_count, solution, repetition):


    def generate_hadamard_series(qubit_count):
        hadamard_series = H
        for i in range(qubit_count - 1):
            hadamard_series = np.kron(hadamard_series, H)
        return hadamard_series

    
    hadamard_series = generate_hadamard_series(qubit_count)

    qml.QubitUnitary(hadamard_series, wires=list(range(qubit_count)))

    qml.PauliX(wires=qubit_count)
    qml.Hadamard(wires=qubit_count)


    for _ in range(repetition):
        qml.MultiControlledX(wires=list(range(qubit_count)) + [qubit_count], control_values=solution)

        qml.QubitUnitary(hadamard_series, wires=list(range(qubit_count)))

        qml.MultiControlledX(wires=list(range(qubit_count)) + [qubit_count], control_values='0' * qubit_count)

        qml.QubitUnitary(hadamard_series, wires=list(range(qubit_count)))

    return qml.probs(wires=list(range(qubit_count)))

for repetition in range(5):
    probabilities = grovers(qubit_count, solution, repetition)
    sqp = get_sqp(probabilities)
    print(sqp)

In [ ]:
shots_list = None

qubit_count = 5
solutions = ["10110", "10111"]

dev = qml.device("default.qubit", wires=qubit_count+1, shots=shots_list)

@qml.qnode(dev)
def grovers_multisolution(qubit_count, solutions, repetition):

    def generate_hadamard_series(qubit_count):
        hadamard_series = H
        for i in range(qubit_count - 1):
            hadamard_series = np.kron(hadamard_series, H)
        return hadamard_series

    hadamard_series = generate_hadamard_series(qubit_count)

    qml.QubitUnitary(hadamard_series, wires=list(range(qubit_count)))

    qml.PauliX(wires=qubit_count)
    qml.Hadamard(wires=qubit_count)

    for _ in range(repetition):
        for solution in solutions:
            qml.MultiControlledX(wires=list(range(qubit_count)) + [qubit_count], control_values=solution)

        qml.QubitUnitary(hadamard_series, wires=list(range(qubit_count)))

        qml.MultiControlledX(wires=list(range(qubit_count)) + [qubit_count], control_values='0' * qubit_count)

        qml.QubitUnitary(hadamard_series, wires=list(range(qubit_count)))

    return qml.probs(wires=list(range(qubit_count)))

for repetition in range(5):
    probabilities = grovers_multisolution(qubit_count, solutions, repetition)
    sqp = get_sqp(probabilities)
    print(sqp)